In [ ]:
import sys
import re
import os
from docx import Document
import pandas as pd
from IPython.display import display, HTML

result = []

def openfile(name):
    d = Document(name)
    return d

def process(doc, filename):
    result = {
        "file_name": filename,
        "data": []
    }
    all_p = doc.paragraphs
    num_re = re.compile("(\d{1,2})\.")

    num = 0
    current_index = -1
    # rrr = map(lambda x: re.match(num_re, x.text), all_p)
    start = False
    for p in all_p:
        num_re = re.compile("(\d{1,2})\.")
        match = re.match(num_re, p.text)

        if match:
            start = True
            num = match.group(1)
            result["data"].append({"num": num, "content": ""})
            current_index = len(result["data"]) - 1
        else:
            if not start:
                pass
            else:
                result["data"][current_index]["content"] += "\n" + p.text

    return result

def query_keyword(result, q):
    return [r for r in result["data"] if q in r["content"]] 

def query_keywordd(result, q):
    rr = {}
    
    for file in result:
        # rr.append({"filename": })
        # rr[file["file_name"]] = [sentence for sentence in file["data"] if q in sentence["content"]]
        rr[file["file_name"]] = [sentence for sentence in file["data"] if re.search(q, sentence["content"])]

    return rr

def process_path(p):
    for filename in os.listdir(p):
        if not filename.startswith("~") and (filename.endswith(".docx") or filename.endswith(".doc")):
            result.append(process(openfile(filename), filename))
    return result

def search(path, query_word):
    """
    path: <Str>
        此參數為所有語料.docx檔的資料夾路徑
        給路徑之後，將會去爬梳該路徑底下所有的.docx檔
    query_word: <Str>
        要搜索的字串，此搜索字串會先丟入re.compile()
        因此可以接收regex string
        例如此參數可以輸入"(LOC|INS)"
        則可以給出所有標有LOC或INS的語料。
    """
    result = process_path(path)
#     query_word = sys.argv[2]

    # if '"' in query_word:
    pattern = re.compile(query_word)

    r = query_keywordd(result, pattern)
    for k, v in r.items():
        print(f"<< {k} >>")
        if len(v) == 0:
            print("本週無相關資料")
        else:
            for item in v:
                n = item['num']
                c = item['content']
                cc = c.split('\n')
                cc = list(filter(lambda x: x!='', cc))
                display(pd.DataFrame([cc[0].split(), cc[1].split(), cc[2].split()]))
                print("[英文翻譯]")
                print([ccc for ccc in cc if ccc.startswith("#e")][0])
                print("[中文翻譯]")
                print([ccc for ccc in cc if ccc.startswith("#c")][0])
                print("[註釋]")
                print([ccc for ccc in cc if ccc.startswith("#n")][0])
                # print(f"{n}:\n{c}")


## 主要搜尋function

### `search()`函數接收兩個參數

- `path` : 所有語料的.docx檔所在路徑
- `query_string` : 所要搜尋的字串，此參數會先丟入`re.compile()`，所以可以使用regex string

### 使用範例
假設我的三個語料.docx檔在 `/users/Don/corpus` 資料夾底下，並且我想找出所有含有`LOC`或`NOM`標記的語料，所以我可以下: 

> `search("/users/Don/corpus", "(NOM|LOC)")`

假設我的三個語料.docx檔與此.ipynb檔同個資料夾，並且我想找出所有含有`CAUS`標記的語料，所以我可以下: 

> `search(".", "CAUS")`

In [ ]:
search('.', '(LOC|INS)')